# TP 8 : Diagnostic de la régression : analyse des observations
Le but du TP est de réaliser un diagnostic de régression général et via l'observation des variables. Ce Tp vous permettra d'implémenter le calcul du $R^2$, ainsi que des coefficients de levier et de contribution.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Ex. 1 — Détection de points aberrants dans la régression
On considère les n = 11 observations suivantes :

i| 1| 2 |3| 4| 5| 6| 7| 8| 9 |10| 11
---|---|---|---|---|---|---|---|---|---|---|---
x | -1.0782 |0.0838| 0.1524 |0.2290| 0.4427| 0.4505| 0.5383| 0.8258| 0.9133| 0.9961| 3.0597
y| -0.8307| 0.1538 |0.0926| 0.2336| 0.3903| 0.1005| 0.4812 |0.6595| 0.7175| 0.7648 |0.3519

Les observations $y$ sont liées aux $x$ à travers le modèle linéaire :
$y_i = ax_i + b + \epsilon_i \text{, avec    } i = 1, n$
où les $\epsilon_i$ sont des variables aléatoires i.i.d. distribuées selon une loi normale centrée de variance
$\sigma^2$ inconnue et $(a, b)$ sont deux paramètres inconnus.

1. Effectuons les calculs liés au modèle de régression et à son diagnostic.

    a) construire une matrice $X$ et un vecteur $y$ contenant les données de la régression que l’on souhaite effectuer.

In [2]:
Data = np.array([
    [1, -1.0782, -.8307],
    [2, 0.0838, 0.1538],
    [3, 0.1524, 0.0926],
    [4, 0.2290, 0.2336],
    [5, 0.4427, 0.3903],
    [6, 0.4505, 0.1005],
    [7, 0.5383, 0.4812],
    [8, 0.8258, 0.6595],
    [9, 0.9133, 0.7175],
    [10, 0.9961, 0.7648],
    [11, 3.0597, 0.3519]
])
x = np.array([-1.0782, 0.0838,0.1524,  0.2290,0.4427,0.4505,0.5383,0.8258,0.9133,0.9961, 3.0597]).T
y = np.array([-.8307,0.1538,0.0926, 0.2336, 0.3903, 0.1005, 0.4812, 0.6595, 0.7175, 0.7648, 0.3519]).T

a) Calculer l’estimation de $a$ et $b$ sens des moindres carrés

b) Plotter $x$ et $y$, ainsi que votre droite de régression.

c) Calculer des résidus et une estimation non biaisée de la variance des erreurs $\sigma^2$.

d) Évaluer la qualité globale de la régression

e) Visualiser les résidus afin de vérifier leur conformité au modèle.

f) Calculer le levier de chaque observation

D'après le cours : 
$$
H = X (X^\top X)^{-1}X^\top
$$
les leviers de chaque point sont les $H_{ii}$
 on peut aussi écrire : 
 $$
 H = X G
 $$
 avec 
 $$
 G = (X^\top X)^{-1}X^\top
 $$
 La matrice $G$ est donc aussi solution des $n$ systèmes linéaires : 
 $$
 (X^\top X) \; G = X^\top
 $$

In [9]:
H = X@np.linalg.inv(X.T@X)@X.T
H = X@np.linalg.solve(X.T@X,X.T)  # plus stable et plus rapide
h = np.diag(H)
h

array([0.37789407, 0.1181502 , 0.11140572, 0.10500641, 0.09346591,
       0.09322048, 0.09131189, 0.09604114, 0.1008192 , 0.10677539,
       0.70590959])

g) Calculer la contribution de chaque observation

d'après le cours 
$$
    c_i = \frac{H_{ii}}{(1 - H_{ii})^2}\frac{\hat \varepsilon_i^2}{p s^2}
$$

h) Visualisez tous ces résultats dans un tableau contenant le vecteur des observations
$x$, le vecteur des réponses $y$, les prédiction du modèle, les résidus, les leviers, les
contributions et une variable binaire repérant les points dont la contribution dépasse
$4/n$.

In [11]:
Res = np.array([x, y, z, e, h, c , c>(4/n)])
print('-------------------------------------------------------')
print('      x      y      z     e      h      c   Aberrant ?')
print('-------------------------------------------------------')
with np.printoptions(precision=3, suppress=True):
    print(Res.T)

-------------------------------------------------------
      x      y      z     e      h      c   Aberrant ?
-------------------------------------------------------
[[-1.078 -0.831 -0.164 -0.666  0.378  1.562  1.   ]
 [ 0.084  0.154  0.145  0.008  0.118  0.     0.   ]
 [ 0.152  0.093  0.164 -0.071  0.111  0.003  0.   ]
 [ 0.229  0.234  0.184  0.05   0.105  0.001  0.   ]
 [ 0.443  0.39   0.241  0.149  0.093  0.009  0.   ]
 [ 0.451  0.101  0.243 -0.143  0.093  0.008  0.   ]
 [ 0.538  0.481  0.266  0.215  0.091  0.018  0.   ]
 [ 0.826  0.659  0.343  0.316  0.096  0.042  0.   ]
 [ 0.913  0.718  0.366  0.351  0.101  0.055  0.   ]
 [ 0.996  0.765  0.388  0.376  0.107  0.068  0.   ]
 [ 3.06   0.352  0.938 -0.586  0.706 10.109  1.   ]]


2. Nous allons maintenant « consolider » ces résultats en écrivant une fonction réutilisable (et par la même nous abstraire du copier/coller et de ces effets néfastes).
    
a) Écrire une fonction Python ma_reg permettant d’intégrer les calculs fait précédemment. Votre fonction prendra la matrice $X$ et le vecteur $y$ comme paramètres d’entrée et rendra :
   
- l’estimation des paramètres de la régression sens des moindres carrés,
- une estimation de la variance des erreurs $\sigma^2$,
- le coefficient de détermination $R^2$
- une matrice n × 3 contenants des éléments permettant de réaliser le diagnostic de la régression à savoir :
    - les résidus,
    - le levier de chaque observation,
    - la contribution de chaque observation.

```python
def ma_reg(X,y): 
    
    return , , ,
```

c) Ecrire un script Python permettant de tester cette fonction. Corriger votre dataset pour éliminer un ou plusieurs points suspects

Sans le dernier point, la regression est bien meilleure

d) Que se passe t’il si l’on exécute les instruction suivantes ?

```python
yp = X
a, s2, R2, diagd = ma_reg(X,yp)
```

e) Ajouter au début de la fonction ma_reg un test permettant de vérifier que les dimensions de la matrice $X$ et du vecteur $y$ sont bien conformes

3\. Effectuer le diagnostic de la régression
    
a) trouver une observation avec un résidu important et un faible levier

b) trouver une observation avec un faible résidu et un fort levier
    
   

c) trouver une observation avec un fort résidu et un levier important. Vérifier que vous avez bien détecté qu’il s’agit d’un point aberrant.
    
 

  
d) Après avoir éliminé le point aberrant, refaire toutes les étapes de la régression

a) trouver une observation avec un résidu important et un faible levier
    


b) trouver une observation avec un faible résidu et un fort levier
   

## Ex. 2 — Modèle mal spécifié
On considère les n = 11 observations suivantes :

x| 1.00 |1.50| 2.00| 2.50| 3.00| 3.50| 4.00| 4.50| 5.00| 5.50| 6.0000
---|---|---|---|---|---|---|---|---|---|---|---|
y| 5.10| 4.56| 4.85| 7.48| 9.10| 11.65| 14.87| 19.20| 24.23| 32.10| 39.80

1. Effectuer le régression linéaire entre x et y et déterminer le coefficient de corrélation et visualiser les résidus. Ces résidus sont-ils conformes aux hypothèses de la régression ?

In [21]:
D = np.array([[1, 5.1],
[1.5, 4.56],
[2, 4.85],
[2.5, 7.48],
[3, 9.1],
[3.5, 11.65],
[4, 14.87],
[4.5, 19.2],
[5, 24.23],
[5.5, 32.1],
[6, 39.8]])

x = D[:,0]
y = D[:,1]

2\. Effectuer la régression polynomiale (degré 2) entre x et y et déterminer le coefficient de détermination. La régression polynomiale consiste à faire l’hypothèse que les observations $y$ sont liées aux $x$ à travers le modèle quadratique :

$y_i = a_0 + a_1x_i + a_2x^2_i + \epsilon_i\text{, avec } i = 1, n$

où les $\epsilon_i$ sont des variables aléatoires i.i.d. distribuées selon une loi normale centrée de
variance $\sigma^2$ inconnue et $(a_0, a_1, a_2)$ sont trois paramètres inconnus.

3\. Visualiser les résidus de la régression et vérifier qu’ils vérifient bien les hypothèses de la régression.

4\. Comparez les coefficients de détermination des deux régressions.